In [1]:
import requests
import json
from IPython.display import Markdown

In [2]:
def get_completion(prompt):
    headers = {'Content-Type': 'application/json'}
    data = {"prompt": prompt, "history": []}
    response = requests.post(url='http://127.0.0.1:8000', headers=headers, data=json.dumps(data))
    return response.json()['response']

get_completion("你好")

'你好👋!我是人工智能助手 ChatGLM-6B,很高兴见到你,欢迎问我任何问题。'

# 迭代式提示开发

&emsp;&emsp;当使用 LLM 构建应用程序时，我从来没有在第一次尝试中就成功使用最终应用程序中所需的 Prompt。但这并不重要，只要您有一个好的迭代过程来不断改进您的 Prompt，那么你就能够得到一个适合任务的 Prompt。我认为在提示方面，第一次成功的几率可能会高一些，但正如上所说，第一个提示是否有效并不重要。最重要的是为您的应用程序找到有效提示的过程。

&emsp;&emsp;因此，在本章中，我们将以从产品说明书中生成营销文案这一示例，展示一些框架，以提示你思考如何迭代地分析和完善你的 Prompt。

&emsp;&emsp;如果您之前与我一起上过机器学习课程，您可能见过我使用的一张图表，说明了机器学习开发的流程。通常是先有一个想法，然后再实现它：编写代码，获取数据，训练模型，这会给您一个实验结果。然后您可以查看输出结果，进行错误分析，找出它在哪里起作用或不起作用，甚至可以更改您想要解决的问题的确切思路或方法，然后更改实现并运行另一个实验等等，反复迭代，以获得有效的机器学习模型。在编写 Prompt 以使用 LLM 开发应用程序时，这个过程可能非常相似，您有一个关于要完成的任务的想法，可以尝试编写第一个 Prompt，满足上一章说过的两个原则：清晰明确，并且给系统足够的时间思考。然后您可以运行它并查看结果。如果第一次效果不好，那么迭代的过程就是找出为什么指令不够清晰或为什么没有给算法足够的时间思考，以便改进想法、改进提示等等，循环多次，直到找到适合您的应用程序的 Prompt。 

## 任务——从产品说明书生成一份营销产品描述 

这里有一个椅子的产品说明书，描述说它是一个中世纪灵感家族的一部分，讨论了构造、尺寸、椅子选项、材料等等，产地是意大利。假设您想要使用这份说明书帮助营销团队为在线零售网站撰写营销式描述

In [3]:
# 示例：产品说明书
fact_sheet_chair = """
概述

    美丽的中世纪风格办公家具系列的一部分，包括文件柜、办公桌、书柜、会议桌等。
    多种外壳颜色和底座涂层可选。
    可选塑料前后靠背装饰（SWC-100）或10种面料和6种皮革的全面装饰（SWC-110）。
    底座涂层选项为：不锈钢、哑光黑色、光泽白色或铬。
    椅子可带或不带扶手。
    适用于家庭或商业场所。
    符合合同使用资格。

结构

    五个轮子的塑料涂层铝底座。
    气动椅子调节，方便升降。

尺寸

    宽度53厘米|20.87英寸
    深度51厘米|20.08英寸
    高度80厘米|31.50英寸
    座椅高度44厘米|17.32英寸
    座椅深度41厘米|16.14英寸

选项

    软地板或硬地板滚轮选项。
    两种座椅泡沫密度可选：中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）。
    无扶手或8个位置PU扶手。

材料
外壳底座滑动件

    改性尼龙PA6/PA66涂层的铸铝。
    外壳厚度：10毫米。
    座椅
    HD36泡沫

原产国

    意大利
"""

In [5]:
# 提示：基于说明书创建营销描述
prompt = f"""
你的任务是帮助营销团队基于技术说明书创建一个产品的营销描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

技术说明: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

产品描述:

这是一个美丽的中世纪风格办公家具系列,包括文件柜、办公桌、书柜、会议桌等。该系列采用高品质材料制成,具有多种颜色和涂层选项,包括不锈钢、哑光黑色、光泽白色或铬。该系列还可选塑料前后靠背装饰(SWC-100)或10种面料和6种皮革的全面装饰(SWC-110)。椅子可带或不带扶手,适用于家庭或商业场所,并具有气动椅子调节功能,方便升降。该系列的尺寸包括宽度53厘米|20.87英寸、深度51厘米|20.08英寸、高度80厘米|31.50英寸和座椅高度44厘米|17.32英寸、座椅深度41厘米|16.14英寸。此外,该系列还提供不同的滚轮选项和泡沫密度选择,以及无扶手或8个位置PU扶手的选择。原产国为意大利。


### 问题一：生成文本太长

它似乎很好地写了一个描述，介绍了一个惊人的中世纪灵感办公椅，很好地完成了要求，即从技术说明书开始编写产品描述。但是当我看到这个时，我会觉得这个太长了。

所以我有了一个想法。我写了一个提示，得到了结果。但是我对它不是很满意，因为它太长了，所以我会澄清我的提示，并说最多使用50个字。

因此，我通过要求它限制生成文本长度来解决这一问题

In [9]:
# 优化后的 Prompt，要求生成描述不多于 50 词
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

使用最多50个字。

技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

美丽的中世纪风格办公家具系列,包括文件柜、办公桌、书柜、会议桌等。多种外壳颜色和底座涂层可选。可选塑料前后靠背装饰(SWC-100)或10种面料和6种皮革的全面装饰(SWC-110)。底座涂层选项为:不锈钢、哑光黑色、光泽白色或铬。椅子可带或不带扶手。适用于家庭或商业场所。符合合同使用资格。

结构:五个轮子的塑料涂层铝底座。气动椅子调节,方便升降。

尺寸:宽度53厘米|20.87英寸

深度51厘米|20.08英寸

高度80厘米|31.50英寸

座椅高度44厘米|17.32英寸

座椅深度41厘米|16.14英寸

材料:外壳底座滑动件

改性尼龙PA6/PA66涂层的铸铝。

外壳厚度:10毫米。

座椅:HD36泡沫

原产国:意大利


In [10]:
len(response)

327

注：很明显chatglm对字数的控制很差，特别是遇见长文本的时候。 

## 问题二：文本关注在错误的细节上

我们会发现的第二个问题是，这个网站并不是直接向消费者销售，它实际上旨在向家具零售商销售家具，他们会更关心椅子的技术细节和材料。在这种情况下，你可以修改这个提示，让它更精确地描述椅子的技术细节。

解决方法：要求它专注于与目标受众相关的方面。

In [12]:
# 优化后的 Prompt，说明面向对象，应具有什么性质且侧重于什么方面
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

使用最多50个单词。

技术规格： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

产品描述:

美丽的中世纪风格办公家具系列的一部分,包括文件柜、办公桌、书柜、会议桌等。该系列有多种外壳颜色和底座涂层可选,可选塑料前后靠背装饰(SWC-100)或10种面料和6种皮革的全面装饰(SWC-110),底座涂层选项为不锈钢、哑光黑色、光泽白色或铬。椅子可带或不带扶手,适用于家庭或商业场所,符合合同使用资格。 

结构:该系列采用五个轮子的塑料涂层铝底座,气动椅子调节,方便升降。尺寸该系列有不同宽度、深度、高度和座椅高度、座椅深度可选。材料外壳底座滑动件采用改性尼龙PA6/PA66涂层的铸铝,外壳厚度为10毫米,座椅采用HD36泡沫。原产国意大利。


In [13]:
# 更进一步
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

使用最多50个单词。

技术规格： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

产品描述:

这款美丽的中世纪风格办公家具系列,由意大利原产,包括文件柜、办公桌、书柜、会议桌等。

有多种外壳颜色和底座涂层可选。可以选择塑料前后靠背装饰(SWC-100)或10种面料和6种皮革的全面装饰(SWC-110)。

底座涂层选项为:不锈钢、哑光黑色、光泽白色或铬。椅子可带或不带扶手。

适用于家庭或商业场所,符合合同使用资格。

结构采用五个轮子的塑料涂层铝底座,气动椅子调节,方便升降。

尺寸包括宽度53厘米|20.87英寸、深度51厘米|20.08英寸、高度80厘米|31.50英寸和座椅高度44厘米|17.32英寸、座椅深度41厘米|16.14英寸。

材料采用改性尼龙PA6/PA66涂层的铸铝外壳和座椅泡沫,其中有两种选项:中等(1.8磅/立方英尺)或高(2.8磅/立方英尺)。

原产国意大利。


注：明显它遗漏了**产品ID：SWC-100/SWC-110**

## 问题三：需要一个表格形式的描述

以上是许多开发人员通常会经历的迭代提示开发的简短示例。我的建议是，像上一章中所演示的那样，Prompt 应该保持清晰和明确，并在必要时给模型一些思考时间。在这些要求的基础上，通常值得首先尝试编写 Prompt ，看看会发生什么，然后从那里开始迭代地完善 Prompt，以逐渐接近所需的结果。因此，许多成功的Prompt都是通过这种迭代过程得出的。我将向您展示一个更复杂的提示示例，可能会让您对ChatGPT的能力有更深入的了解。

这里我添加了一些额外的说明，要求它抽取信息并组织成表格，并指定表格的列、表名和格式，还要求它将所有内容格式化为可以在网页使用的 HTML。

In [16]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

在描述之后，包括一个表格，提供产品的尺寸。表格应该有两列。第一列包括尺寸的名称。第二列只包括英寸的测量值。

给表格命名为“产品尺寸”。

将所有内容格式化为可用于网站的HTML格式。将描述放在<div>元素中。

技术规格：```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

产品描述:

产品名称:办公家具系列

产品ID:WC-101

概述:

该系列是美丽的中世纪风格办公家具,包括文件柜、办公桌、书柜、会议桌等。有多种外壳颜色和底座涂层可选,包括不锈钢、哑光黑色、光泽白色或铬。椅子可带或不带扶手,适用于家庭或商业场所,符合合同使用资格。

结构:

该系列由五个轮子的塑料涂层铝底座组成,气动椅子调节,方便升降。外壳和座椅由改性尼龙PA6/PA66涂层的铸铝组成,厚度为10毫米。

尺寸:

宽度53厘米|20.87英寸

深度51厘米|20.08英寸

高度80厘米|31.50英寸

座椅高度44厘米|17.32英寸

座椅深度41厘米|16.14英寸

选项:

软地板或硬地板滚轮选项

两种座椅泡沫密度可选:中等(1.8磅/立方英尺)或高(2.8磅/立方英尺)

无扶手或8个位置PU扶手

材料:

外壳底座滑动件

改性尼龙PA6/PA66涂层的铸铝

外壳厚度:10毫米

座椅

HD36泡沫

原产国:

意大利

以上产品描述已格式化为可用于网站的HTML格式,并将其放在<div>元素中。请注意,产品ID和产品描述中的产品名称和原产国应该匹配。


In [17]:
# 表格是以 HTML 格式呈现的，加载出来
from IPython.display import display, HTML

display(HTML(response))

注：很明显它完全没有想起来还有\<div\>网页标签这回事 

# 总结 

chatglm在面对长文本的时候，容易无法把内容全部提炼。